### Deliverable 1: Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN","NAME"],inplace=True)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
application_types=application_df["APPLICATION_TYPE"].value_counts()

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application=application_types[application_types <200].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count=application_df["CLASSIFICATION"].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class=classification_count[classification_count<1000].index.tolist()

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Dropping noisy data after reviewing the value count and the feature importance
print(application_df["SPECIAL_CONSIDERATIONS"].value_counts())
print(application_df["STATUS"].value_counts())
application_df.drop(columns=["SPECIAL_CONSIDERATIONS","STATUS"],inplace=True)

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64
1    34294
0        5
Name: STATUS, dtype: int64


In [9]:
# Generate our categorical variable lists
application_cat=application_df.dtypes[application_df.dtypes=="object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df=pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
combined_df=application_df.join(encode_df).drop(columns=application_cat)
combined_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
X=combined_df.drop("IS_SUCCESSFUL",axis=1)
y=combined_df["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(X_train_scaled,y_train)

RandomForestClassifier()

In [15]:
importances=rfc.feature_importances_
sorted(zip(importances,X.columns),reverse=True)

[(0.4045098123227089, 'ASK_AMT'),
 (0.13009341841378552, 'AFFILIATION_CompanySponsored'),
 (0.116942931540457, 'AFFILIATION_Independent'),
 (0.028554505347999408, 'APPLICATION_TYPE_T5'),
 (0.026752871668639194, 'ORGANIZATION_Association'),
 (0.02358075489456761, 'APPLICATION_TYPE_T10'),
 (0.022455361695903835, 'ORGANIZATION_Trust'),
 (0.02196606952135934, 'CLASSIFICATION_C2100'),
 (0.019524283568170955, 'CLASSIFICATION_Other'),
 (0.0193000325804886, 'CLASSIFICATION_C2000'),
 (0.018560952092888192, 'APPLICATION_TYPE_T4'),
 (0.01770513177882911, 'APPLICATION_TYPE_T19'),
 (0.016505215308505603, 'APPLICATION_TYPE_T3'),
 (0.014751246256286347, 'CLASSIFICATION_C1000'),
 (0.012395979003098676, 'USE_CASE_ProductDev'),
 (0.011948369157627427, 'APPLICATION_TYPE_T6'),
 (0.010106303104721555, 'CLASSIFICATION_C1200'),
 (0.009271259302902238, 'USE_CASE_Preservation'),
 (0.008665707949247536, 'INCOME_AMT_0'),
 (0.007419549013809179, 'INCOME_AMT_25000-99999'),
 (0.0073265744959719594, 'CLASSIFICATION_

### Deliverable 2: Compile, Train and Evaluate the Model

In [16]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("checkpoints/Deliverable3",exist_ok=True)
checkpoint_path="checkpoints/Deliverable3/weights.{epoch:02d}.hdf5"

In [123]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features=len(X_train_scaled[0])
hidden_nodes_layer1=100
hidden_nodes_layer2=50
hidden_nodes_layer3=20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,activation="relu",input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2,activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3,activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_90 (Dense)            (None, 100)               4100      
                                                                 
 dense_91 (Dense)            (None, 50)                5050      
                                                                 
 dense_92 (Dense)            (None, 20)                1020      
                                                                 
 dense_93 (Dense)            (None, 1)                 21        
                                                                 
Total params: 10,191
Trainable params: 10,191
Non-trainable params: 0
_________________________________________________________________


In [124]:
# Compile the model
nn.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [52]:
# Callback
Batch_size=59
No_of_steps=int(len(y_train)/Batch_size)
callback=ModelCheckpoint(filepath=checkpoint_path,
                        verbose=1,
                        save_weights_only=True,
                        save_freq=No_of_steps*5)

In [125]:
# Train the model
nn.fit(X_train_scaled,y_train,epochs=50, batch_size=Batch_size, callbacks=[callback])

Epoch 1/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5726 - accuracy: 0.7183
Epoch 2/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5552 - accuracy: 0.7281
Epoch 3/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5523 - accuracy: 0.7299
Epoch 4/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5501 - accuracy: 0.7304
Epoch 5/50
435/436 [============================>.] - ETA: 0s - loss: 0.5494 - accuracy: 0.7306
Epoch 5: saving model to checkpoints/Deliverable3\weights.05.hdf5
436/436 [==============================] - 1s 2ms/step - loss: 0.5495 - accuracy: 0.7305
Epoch 6/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5483 - accuracy: 0.7315
Epoch 7/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7321
Epoch 8/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5470 - accuracy: 0.7333
Epoch 9/50
436/436 [=============================

In [126]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5591 - accuracy: 0.7301 - 412ms/epoch - 2ms/step
Loss: 0.5591232180595398, Accuracy: 0.7301457524299622


In [127]:
nn.save("AlphabetSoupCharity_Optimization.h5")